In [1]:
# !wget https://huggingface.co/datasets/rizerphe/glaive-function-calling-v2-llama/resolve/main/data/train-00000-of-00001-5eaf45abe9bced63.parquet?download=true -O function-call.parquet

In [2]:
# !wget https://huggingface.co/datasets/glaiveai/glaive-function-calling-v2/resolve/main/glaive-function-calling-v2.json?download=true -O call.json

In [17]:
# !wget https://huggingface.co/datasets/glaiveai/glaive-function-calling/resolve/main/glaive_function_calling.json?download=true -O call-v1.json

In [46]:
import json
from tqdm import tqdm

with open('call.json') as fopen:
    data = json.load(fopen)

In [47]:
data[0]['chat']

{'system': 'SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n{\n    "name": "get_exchange_rate",\n    "description": "Get the exchange rate between two currencies",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "base_currency": {\n                "type": "string",\n                "description": "The currency to convert from"\n            },\n            "target_currency": {\n                "type": "string",\n                "description": "The currency to convert to"\n            }\n        },\n        "required": [\n            "base_currency",\n            "target_currency"\n        ]\n    }\n}\n',
 'chat': "USER: Can you book a flight for me from New York to London?\n\n\nASSISTANT: I'm sorry, but I don't have the capability to book flights. My current function allows me to get the exchange rate between two currencies. If you need help with that, feel free to ask! <|endoftext|>\n\n\n"}

In [54]:
all_data = []
for d in tqdm(data):
    function_call = None
    try:
        function_call = d['system'].split('required -')[1]
    except Exception as e:
        continue
    
    conversations = []
    splitted = d['chat'].split('ASSISTANT:')
    conversations.append({
        'role': 'user',
        'content': splitted[0].split('USER:')[1].strip(),
    })
    for s in splitted[1:]:
        s = s.replace('<|endoftext|>', '')
        if 'USER:' in s:
            s = s.split('USER:')
            conversations.append({
                'role': 'assistant',
                'content': s[0].strip(),
            })
            conversations.append({
                'role': 'user',
                'content': s[1].strip(),
            })
        else:
            conversations.append({
                'role': 'assistant',
                'content': s.strip(),
            })
        
    all_data.append({
        'function_call': function_call.strip(),
        'conversations': conversations
    })

100%|███████████████████████████████| 112960/112960 [00:00<00:00, 307104.33it/s]


In [55]:
with open('call-v1.json') as fopen:
    data_v1 = json.load(fopen)

In [56]:
for d in tqdm(data_v1):
    system = d['sample'].split('USER:')[0]
    function_call = None
    try:
        function_call = system.split('needed-')[1].strip()
    except Exception as e:
        continue
    
    conversations = []
    splitted = ('USER: ' + 'USER:'.join(d['sample'].split('USER:')[1:]).strip()).split('ASSISTANT:')
    conversations.append({
        'role': 'user',
        'content': splitted[0].split('USER:')[1].strip(),
    })
    for s in splitted[1:]:
        s = s.replace('<|endoftext|>', '')
        if 'USER:' in s:
            s = s.split('USER:')
            conversations.append({
                'role': 'assistant',
                'content': s[0].strip(),
            })
            conversations.append({
                'role': 'user',
                'content': s[1].strip(),
            })
        else:
            conversations.append({
                'role': 'assistant',
                'content': s.strip(),
            })
        
    all_data.append({
        'function_call': function_call.strip(),
        'conversations': conversations
    })

100%|█████████████████████████████████| 52893/52893 [00:00<00:00, 136932.45it/s]


In [58]:
all_data[-1]

{'function_call': '{\n    "name": "calculate_discount",\n    "description": "Calculate the final price after applying a discount",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "original_price": {\n                "type": "number",\n                "description": "The original price of the item"\n            },\n            "discount_percentage": {\n                "type": "number",\n                "description": "The percentage of discount to be applied"\n            }\n        },\n        "required": [\n            "original_price",\n            "discount_percentage"\n        ]\n    }\n}',
 'conversations': [{'role': 'user',
   'content': 'Can you help me calculate the final price after applying a discount?'},
  {'role': 'assistant',
   'content': 'Sure! I can help you with that. Please provide me with the original price and the discount percentage.'},
  {'role': 'user',
   'content': 'The original price is $100 and the discount percentage is 

In [57]:
len(all_data)

121254

In [59]:
with open('function-call.jsonl', 'w') as fopen:
    for d in all_data:
        fopen.write(f'{json.dumps(d)}\n')

In [60]:
texts = []
for d in all_data:
    for c in d['conversations']:
        if 'functioncall' not in c['content']:
            texts.append(c['content'])
            
len(texts)

551914

In [61]:
texts = [str(t) if isinstance(t, dict) else t for t in texts]
texts = [t for t in texts if len(t)]
texts = list(set(texts))
len(texts)

111056

In [62]:
texts[:10]

["I'm sorry, but I'm unable to assist with booking flights. My current capabilities are limited to calculating loan payments based on the principal amount, interest rate, and term. If you need help with that, feel free to ask!",
 "I need to create an invoice for my client, John Doe. The amount is $5000 and it's due on 2022-12-31.",
 "I'm sorry, but I'm unable to perform external tasks like ordering a pizza. My current capabilities are limited to the functions provided to me. In this case, I can help you calculate the tip amount for a bill.",
 'The current traffic on the Brooklyn Bridge is heavy. It is estimated that it will take around 30 minutes to cross.',
 'Based on the details provided, your estimated insurance premium would be $200 per month.',
 'Here are more details about Hotel ABC:\n- Price: $200 per night\n- Rating: 4.5\n- Amenities: Free Wi-Fi, Swimming Pool, Gym\n- Contact: Phone - +1 123-456-7890, Email - info@hotelabc.com',
 "That's a lot of yen! Thanks for the help.",
 'T

In [67]:
with open('function.text', 'w') as fopen:
    for t in texts:
        fopen.write(f'{json.dumps(t)}\n')

In [73]:
!split -l 50000 -d --additional-suffix=.splitted function.text function.text

In [74]:
!ls function.text*

function.text		  function.text01.splitted
function.text00.splitted  function.text02.splitted


In [72]:
!rm function.text0*.splitted*